In [ ]:
import pandas as pd
import json
import numpy as np
import time
import datetime
from scipy.sparse import csr_matrix
import scipy.stats as stats
from sklearn.metrics.pairwise import pairwise_distances
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import os.path as path
import re

# Постановка задачи

**Задача**

Необходимо построить векторное представление музыкантов так, чтобы наиболее похожие музыка оказались в пространстве признаков как можно ближе друг к другу

**Имеющиеся данные**
1. Данные о пользователях
2. Данные о музыкантах
3. Данные о музыкальных композициях
4. Данные о альбомах
5. Данные о плейлистах пользователей

**Выдвижение гипотезы**

Т.к. в нашем датасете нет информации о реальной "схожести" исполнителей, то для того, чтобы искать эту "схожесть" необходимо поставить гипотезу.

Гипотеза - пользователи в общей своей массе слушают похожих друг на друга исполнителей.\
Придерживаясь данной гипотезы мы можем перейти к задаче рекомендации похожих исполнителей пользователям

**Способ решения задачи**

Задачу поиска похожих исполнителей можно свести к задаче создания content-based рекомендательной системы\
Будем рекомендовать пользователю исполнителей, похожих на тех исполнителей, которых пользователь уже послушал

**Оценка работы моделей**

В нашем датасете отсутствует информация о том, какую оценку пользователь поставил треку, у нас имеются лишь данные о том, какие треки пользователь прослушал.

Таким образом, у нас имеется implicit feedback, но не explicit feedback.

То есть, если пользователь прослушал исполнителя i, но не прослушал исполнителя j, то это не значит, что ему нравится исполнитель i и не нравится исполнитель j, вполне возможно, что пользователь просто не знает про существование исполнителя j.\
Поэтому нам не подходят такие метрики оценивания как MAE, RMSE(например предсказывать, сколько раз пользователь будет слушать треки исполнителя i), которые часто используются в explicit feedback моделях для предсказания оценки, потому что у нас банально нет оценок, а также нет адекватной альтернативы, которую можно использовать в качестве оценки. Можно было бы использовать использовать в качестве target variable, например, время прослушивания треков артиста, или количество прослушиваний треков артиста пользвоателем, но в таком случае точность для каждого пользователя будет сильно зависеть от того, как часто пользователь слушает треки (например, для пользователя, часто прослушивающего треки, будет выгодно предсказывать большее количество треков/времени их прослушивания), в то время как пользователю, редко слушающему музыку, выгодно предсказать, что он вовсе ничего не прослушает. Однако, такие результаты не будут отражать реальную точность результатов

Вместо этого я предлагаю использовать метрики, основанные на ранжировании музыкантов по "вероятности" того, что пользователь прослушает треки данного исполнителя. При этом измерять точность модели я буду на основании того, на каком месте в ранжированном списке исполнителей находятся исполнители, которых пользователь в действительности послушал. Такой способ оценки позволит оценить релевантность рекомендаций вне зависимости от того, как много и как часто пользователь слушает музыку. Чем "ближе" исполнители, которых слушает пользователь, к топу ранжированного списка, тем релевантнее рекомендация.

**Алгоритм решения задачи**
1. Построить векторное представление исполнителей
2. Посчитать расстояние между всеми исполнителями.
3. Выбрать информацию о расстоянии между исполнителями, которых прослушал пользователь, и всеми остальными исполнителями
4. Для всех исполнителей посчитать среднее расстояние от них до исполнителей, которых уже послушал пользователь
5. Отранжировать исполнителей в порядке возрастания среднего расстояния до уже прослушанных исполнителей
6. Посчитать метрику работы модели - среднее относительное место(кванитиль) исполнителей, которых прослушал пользователь, в ранжированном списке исполнителей. Следовательно, чем меньшее среднее место (квантиль) - тем лучше модель рекомендует пользователю исполнителей

При этом вместо абсолютного среднего места будет использоваться именно относительное (то есть среднее место разделить на длину ранжированного списка), т.к. при использовании разных данных(о плейлистах/сессиях/всех прослушиваниях) длины ранжированного списка получались разные (потому что в разных датасетах есть информация о разных исполнителях, не о всех сразу).



При этом метрики расстояния я буду использовать разные, а не только евклидову метрику расстояния

# Загрузка данных

## Список исполнителей

1. MBID
2. Name

In [ ]:
person_data = pd.read_csv('entities\\persons.idomaar', sep = '\t', names = ['person', 'person_id', 'timestamp', 'json_data', 'delete'])

In [ ]:
person_data['json_data'] = person_data['json_data'].apply(json.loads)

In [ ]:
person_data['MBID'] = person_data['json_data'].apply(lambda x: x['MBID'])
person_data['artist_name'] = person_data['json_data'].apply(lambda x: x['name'])

In [ ]:
print(person_data['person'].unique())
print(person_data['timestamp'].unique())

In [ ]:
person_data = person_data.drop(['person', 'timestamp', 'json_data', 'delete'], axis = 'columns')

In [ ]:
person_data.head()

In [ ]:
person_data.to_csv('persons.csv', index = False)

## Список треков

1. Длительность трека
2. Сколько раз прослушали трек
3. MBID
4. Название
5. Артист
6. Альбом
7. Тэги

In [ ]:
tracks_data = pd.read_csv('entities\\tracks.idomaar', sep = '\t', names = ['track', 'track_id',  'timestamp', 'json_data', 'linked_data'])

In [ ]:
tracks_data['json_data'] = tracks_data['json_data'].apply(json.loads)

In [ ]:
tracks_data['duration'] = tracks_data['json_data'].apply(lambda x: x['duration'])
tracks_data['playcount'] = tracks_data['json_data'].apply(lambda x: x['playcount'])
tracks_data['MBID'] = tracks_data['json_data'].apply(lambda x: x['MBID'])
tracks_data['name'] = tracks_data['json_data'].apply(lambda x: x['name'])

In [ ]:
tracks_data['linked_data'] = tracks_data['linked_data'].apply(json.loads)

In [ ]:
tracks_data['artist_id'] = tracks_data['linked_data'].apply(lambda x: x['artists'][0]['id'])
tracks_data['albums'] = tracks_data['linked_data'].apply(lambda x: x['albums'])
tracks_data['tags'] = tracks_data['linked_data'].apply(lambda x: x['tags'])

In [ ]:
tracks_data = tracks_data.drop(['track', 'timestamp', 'json_data', 'linked_data'], axis = 'columns')

In [ ]:
tracks_data.head()

In [ ]:
tracks_data.to_csv('tracks.csv', index = False)

## Плейлисты

In [ ]:
playlist_data = pd.read_csv('entities\\playlist.idomaar', sep = '\t', names = ['playlist', 'playlist_id', 'timestamp', 'json_data', 'playlist_tracks'])

In [ ]:
playlist_data['json_data'] = playlist_data['json_data'].apply(lambda x: re.sub('\"Title\":.*?,(?=\"numtracks\")', '', x))
playlist_data['json_data'] = playlist_data['json_data'].apply(json.loads)

In [ ]:
playlist_data['numtracks'] = playlist_data['json_data'].apply(lambda x: x['numtracks'])
playlist_data['duration'] = playlist_data['json_data'].apply(lambda x: x['duration'])

In [ ]:
playlist_data['playlist_tracks'] = playlist_data['playlist_tracks'].apply(json.loads)

In [ ]:
playlist_data['user_id'] = playlist_data['playlist_tracks'].apply(lambda x: x['subjects'][0]['id'])

In [ ]:
playlist_data['playlist_tracks'] = playlist_data['playlist_tracks'].apply(lambda x: x['objects'])
playlist_data = playlist_data[playlist_data['playlist_tracks'].astype(str) != '[[]]'].reset_index(drop = True)
playlist_data = playlist_data[playlist_data['numtracks']!= 0].reset_index(drop = True)

In [ ]:
playlist_data['playlist_tracks'] = playlist_data['playlist_tracks'].apply(lambda x: list(map(lambda y: y['id'], x)))

In [ ]:
playlist_data = playlist_data.explode('playlist_tracks')
playlist_data = playlist_data.rename(columns = {'playlist_tracks' : 'track_id'})

In [ ]:
playlist_data = playlist_data.drop(['playlist', 'json_data'], axis = 'columns')

In [ ]:
playlist_data.head()

In [ ]:
playlist_data.to_csv('playlists.csv', index = False)

## Список прослушиваний

1. Play time
2. Пользователь
3. Трек

In [ ]:
event_data = pd.read_csv('relations\\events.idomaar', sep = '\t', names = ['event_type', 'event_id',  'timestamp', 'playtime', 'json_data'])

In [ ]:
event_data['json_data'] = event_data['json_data'].apply(json.loads)

In [ ]:
event_data['user_id'] = event_data['json_data'].apply(lambda x: x['subjects'][0]['id'])
event_data['track_id'] = event_data['json_data'].apply(lambda x: x['objects'][0]['id'])

In [ ]:
event_data['playtime'] = event_data['playtime'].apply(lambda x: json.loads(x)['playtime'])

In [ ]:
event_data = event_data.drop(['event_type', 'json_data'], axis = 'columns')

In [ ]:
event_data.head()

In [ ]:
event_data.to_csv('events.csv', index = False)

## Список сессий

In [ ]:
session_data = pd.read_csv('relations\\sessions.idomaar', sep = '\t', names = ['event.session', 'session_id', 'timestamp', 'json_data'])

In [ ]:
session_data = pd.concat([session_data, session_data['json_data'].str.split(' ', expand = True).rename(columns = {0 : 'session_data', 1: 'session_tracks'})], axis = 'columns')

In [ ]:
session_data['session_data'] = session_data['session_data'].apply(json.loads)

In [ ]:
session_data['numtracks'] = session_data['session_data'].apply(lambda x: x['numtracks'])
session_data['playtime'] = session_data['session_data'].apply(lambda x: x['playtime'])

In [ ]:
session_data['session_tracks'] = session_data['session_tracks'].apply(json.loads)

In [ ]:
session_data['user_id'] = session_data['session_tracks'].apply(lambda x: x['subjects'][0]['id'])
session_data['session_tracks'] = session_data['session_tracks'].apply(lambda x: x['objects'])

In [ ]:
session_data = session_data.explode('session_tracks')

In [ ]:
session_data['action_type'] = session_data['session_tracks'].apply(lambda x: x['action'])
session_data['track_id'] = session_data['session_tracks'].apply(lambda x: x['id'])
session_data['playstart'] = session_data['session_tracks'].apply(lambda x: x['playstart'])
session_data['playtime'] = session_data['session_tracks'].apply(lambda x: x['playtime'])
session_data['playratio'] = session_data['session_tracks'].apply(lambda x: x['playratio'])

In [ ]:
session_data = session_data.drop(['event.session', 'json_data', 'session_data', 'session_tracks'], axis = 'columns')

In [ ]:
session_data.head()

In [ ]:
session_data.to_csv('sessions.csv', index = False)

# Обзор и чистка данных

## Tracks

In [ ]:
track_data = pd.read_csv('tracks.csv')

Проверим данные на наличие дубликатов

In [ ]:
duplicated_rows = track_data.duplicated(['track_id', 'artist_id'])
duplicated_rows = duplicated_rows[duplicated_rows == True]

In [ ]:
duplicated_tracks = track_data.loc[duplicated_rows.index, :]['track_id'].values
duplicated_artists = track_data.loc[duplicated_rows.index, :]['artist_id'].values

In [ ]:
len(duplicated_rows)

Таким образом, всего в данных о треках есть 1155824 дубликатов. Рассмотрим их подробнее

In [ ]:
track_data[(track_data['track_id'].isin(duplicated_tracks)) & (track_data['artist_id'].isin(duplicated_artists))].sort_values('artist_id')

Удалим все дубликаты в парах (track_id - artist_id)

In [ ]:
track_data = track_data[(track_data['track_id'].isin(duplicated_tracks) == False) & (track_data['artist_id'].isin(duplicated_artists) == False)]

In [ ]:
track_data = track_data.reset_index(drop = True)

Теперь рассмотрим повторяющиеся треки

In [ ]:
duplicated_tracks = track_data.duplicated('track_id')
duplicated_tracks = duplicated_tracks[duplicated_tracks == True]
duplicated_tracks = track_data.loc[duplicated_tracks.index, :]['track_id'].values

In [ ]:
duplicated_rows = track_data[track_data['track_id'].isin(duplicated_tracks)].sort_values('track_id')

In [ ]:
duplicated_rows

Подробнее рассмотрим данных исполнителей

In [ ]:
person_data = pd.read_csv('persons.csv')

In [ ]:
person_data[person_data['person_id'].isin(duplicated_rows['artist_id'].values)]

На самом деле исполнители с разными id одни и те же исполнители, у которых просто немного отличаются названия групп/имена исполнителей. Adventures+Of+Stevie+V тоже самое, что и The+Adventures+Of+Stevie+V. Объединим в нашем датасете этих исполнителей.

In [ ]:
track_data['artist_id'] = track_data['artist_id'].replace({468413: 11173, 168789: 168790, 416502: 416503})

Рассмотрим частоту встречаемости названий треков

In [ ]:
track_data['name'].value_counts()

Треки с названием ZZ+Top/_/She+Loves+My+Automobile  встречаются слишком часто. Рассмотрим их подробнее

In [ ]:
track_data[track_data['name'] == 'ZZ+Top/_/She+Loves+My+Automobile']

Рассмотрим несколько исполнителей из датасета выше

In [ ]:
person_data[person_data['person_id'].isin([508184, 224835, 440240, 595138, 595139])]

Исполнители действительно разные, но названия треков одинаковые. Скорее всего это просто ошибочные названия треков

Удалим появившиесся дубликаты в паррах track_id - artist_id

In [ ]:
track_data = track_data.drop_duplicates(['track_id', 'artist_id'])

Рассмотрим пропущенные значения в колонке duration

In [ ]:
track_data[(track_data['duration'].isna()) | (track_data['duration'] == -1)]

В большинстве треков пропущены данные о длительности, так что я не буду использовать эту переменную

Сохраним отфильтрованный датасет

In [ ]:
track_data.to_csv('tracks_modified.csv', index = False)

## Events

In [ ]:
event_data = pd.read_csv('events.csv')

Рассмотрим информацию о дате прослушивания

In [ ]:
dates = event_data['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
_ = plt.figure(figsize = (10, 10))
_ = plt.title('Распределение количества прослушанных треков во времени')
_ = dates.hist(bins = 200, grid = False)
_ = plt.xlabel('date')

Распределение равномерное, с выбросом ближе к концу выборки. Будем учитывать это при оценке работы моделей (будем стараться избегать конца выборки, т.к. там данные не следуют общей тенденции)

Рассмотрим информацию о времени прослушивания

In [ ]:
_ = plt.figure(figsize = (10,10))
_ = plt.title('99 квантиль распределения длительности трека')
_ = sns.distplot(event_data[event_data['playtime']<np.quantile(event_data['playtime'], 0.99)]['playtime'])

Распределение прослушивания треков выглядит как нормальное, однако присутствует длинный хвост справа\
Также в левой части распределения наблюдается выброс. Рассмотрим его подробнее

In [ ]:
event_data['playtime'].value_counts()

У большого количества треков длительность прослушивания 0 или -1. Удалим наблюдения, в которых длительность прослушивания менее 10 секунд, т.к. можно предположить, что польователь просто скипал эти треки

In [ ]:
event_data = event_data[event_data['playtime']>=10]

Сохраним отфильтрованные данные

In [ ]:
event_data.to_csv('events_modified.csv', index = False)

## Sessions

In [ ]:
session_data = pd.read_csv('sessions.csv')

Рассмотрим тип действия во время сессии

In [ ]:
session_data['action_type'].value_counts(dropna = False)

In [ ]:
session_data[session_data['action_type'] == 'skip']['playtime'].sort_values()

Удалим из сессий те треки, которые пользователь скипнул

In [ ]:
session_data = session_data[session_data['action_type'] != 'skip']

In [ ]:
session_data[session_data['action_type'].isna()]['playtime'].sort_values()

Если нет информации о типе действия, то нет и информации о длителньости прослушивания трека. Также удалим такие наблюдения

In [ ]:
session_data = session_data[session_data['action_type'].isna() == False]

In [ ]:
sns.distplot(session_data['timestamp'])

Распределение времени сессий такое же, как и у времени прослушивания треков

Сохраним полученный датасет

In [ ]:
session_data.to_csv('sessions_modified.csv', index = False)

# Валидация моделей

Общий алгоритм оценки работы моделей:

1. Разбить данные по времени на 2 подвыборки - train и test (длительность test period = 60 дней, начало тестового периода: 07/10/2014)
2. Выбрать тех пользователей, у которых есть оценка в обоих подвыборках
3. Рандомно выбрать из них 100 пользователей
4. Для каждого пользователя считать метрику близости между исполнителями, которых прослушал пользователь, и всеми остальными исполнителями
5. Сортировать артистов по возрастанию среднего метрики близости от уже прослушанных исполнителей
6. Считать среднее относительное место исполнителей, которых прослушал пользователь во время test периода. Меньше место - лучше результат модели

In [ ]:
## Функция подсчета количества треков исполнителя для каждого пользователя/плейлиста/сессии
def count_agg(event_data, track_data, second_var, agg_col, fun):
    merged_data = event_data.merge(track_data, on = 'track_id')
    user_artist_count = merged_data.groupby([second_var, 'artist_id']).agg({agg_col : fun}).rename(columns = {agg_col : 'stat'}).reset_index()
    
    return(user_artist_count)

In [ ]:
## Добавление индексов для исполнителя и второй переменной (пользователь, сессия, плейлист и т.д.)
def add_idx(data, second_var):
    indexes_user = pd.DataFrame({
        '{}'.format(second_var) : data[second_var].unique(),
        '{}_matrix'.format(second_var) : list(range(0, len(data[second_var].unique())))
    }
    )
    indexes_artist = pd.DataFrame({
        'artist_id' : data['artist_id'].unique(),
        'artist_id_matrix' : list(range(0, len(data['artist_id'].unique())))
    }
    )
    data = data.merge(indexes_user).merge(indexes_artist)
    return data

In [ ]:
## Разбиваем на train и test по времени и выбираем пользователей, для которых будем рекомендовать исполнителей
def traintest_split(event_data, test_start = 1405000000, test_period = 5184000):
    train_events = event_data[event_data['timestamp'] <= test_start]
    test_events = event_data[(event_data['timestamp'] > test_start) & (event_data['timestamp'] < (test_start+test_period))]
        
    return (train_events, test_events)

In [ ]:
## Разбиваем на train и test по времени и выбираем пользователей, для которых будет рекомендовать исполнителей
def select_users(event_data, test_start = 1405000000, test_period = 5184000, num_users = 100, seed = 599_000):
    train_events = event_data[event_data['timestamp'] <= test_start]
    train_events = count_agg(train_events, track_data, 'user_id', 'event_id', 'count')
    
    test_events = event_data[(event_data['timestamp'] > test_start) & (event_data['timestamp'] < (test_start+test_period))]
    test_events = count_agg(test_events, track_data, 'user_id', 'event_id', 'count')
    
    train_users = train_events[train_events['stat'] > 5]['user_id'].values
    test_users = test_events[train_events['stat'] > 5]['user_id'].values
    
    aval_users = np.intersect1d(train_users, test_users)
    np.random.seed(seed)
    sampled_users = np.random.choice(aval_users, num_users)
    
        
    return(sampled_users)

In [ ]:
## Создание матрицы исполнители - пользователи/сессии/плейлисты и тд.
def creating_matrix(data, col_data, value):
    col = data[col_data].values
    row = data['artist_id_matrix'].values
    data = data[value].values
    matrix = csr_matrix((data, (row, col)))
    return matrix

In [ ]:
## Выбрать исполнителей, похожих на исполнителей, которых пользователь уже слушал, на основе матрицы train_matrix
def predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, metric, inverse = False, verbose = True):
    global similarities
    all_metrics = []
    num_train_artists = []
    num_test_artists = []
    
    for user in sampled_users: ## Для каждого пользователя
        start = time.time()
        
        ## Выбираем информацию о прослушиваниях исполнителей для конкретного пользователя для тренировочного и тестового периодов
        user_train_data = train_data[train_data['user_id'] == user]
        user_train_artists = user_train_data['artist_id'].values
        user_test_data = test_data[test_data['user_id'] == user]
        user_test_artists = user_test_data['artist_id'].values
        
        ## Переводим id исполнителей в номера строк матрицы информации об исполнителях
        user_train_artists = matrix_id[matrix_id['artist_id'].isin(user_train_artists)]['artist_id_matrix'].unique()
        user_test_artists = matrix_id[matrix_id['artist_id'].isin(user_test_artists)]['artist_id_matrix'].unique()
        
        ## Создаем пустой векторр схожести исполнителей
        user_similarity_matrix = np.empty(shape = (train_matrix.shape[0], 1))
        
        ## Для каждого исполнителя из списка исполнителей, которых прослушал пользователь во время train периода
        for artist in user_train_artists:
            
            ## Считаем схожесть между исполнителем и всеми остальными исполнителями из матрицы информации об исполнителях
            artist_similarity = pairwise_distances(train_matrix, train_matrix[artist, :], metric = metric)
            
            ## Развернуть ли метрику схожести или нет (например с евклидовой метрикой - больше расстояние, разворачиваем, значением метрики становится меньше)
            if inverse == True:
                artist_similarity = 1/(1+artist_similarity)
            
            ## Добавляем столбец схожестимежду исполнителем и всеми остальными исполнителями
            user_similarity_matrix = np.concatenate((user_similarity_matrix, artist_similarity), axis = 1)
        
        ## Считаем среднюю схожесть между тренировочными исполнителями и всеми остальными
        similarities = np.mean(user_similarity_matrix, axis = 1)
        
        ## Сортируем индексы строк по убыванию значения в этой строке в порядке возрастания (то есть от самых похожих к самым непохожим исполнителям)
        similarities = np.argsort(similarities)  ### argsort по умолчанию сортирует в порядке возрастания 
        
        ## Считаем средний индекс (то есть среднюю позицию в ранжированном списке) исполнителей из тестового периода, которых прослушал пользователь
        positions = np.isin(similarities, user_test_artists)
        positions = np.where(positions == True)
        result = np.mean(positions[0])/len(similarities)
        
        ## Добавляем информацию о метрике и количестве исполнителей в тренировочном и тестовом датасете
        all_metrics.append(result)
        num_train_artists.append(len(user_train_artists))
        num_test_artists.append(len(user_test_artists))  
            
        if verbose:
            print('Metric: {}, Number of train artists: {},Number of test artists: {}, Train time: {}'.format(result, len(user_train_artists), len(user_test_artists), time.time()-start))
    return(all_metrics, sampled_users, num_train_artists, num_test_artists)

In [ ]:
## Добавить результаты в csv файл
def add_results(all_metrics, sampled_users, num_train_artist, num_test_artists, model_name):
    data = pd.DataFrame(np.array([all_metrics, sampled_users, num_train_artist, num_test_artists]).T, columns = pd.MultiIndex.from_product([[model_name], ['scores', 'user', 'num_train_artist', 'num_test_artists']]))
    if path.exists('results.csv'):
        results_loaded = pd.read_csv('results.csv', header=[0,1])
        data = pd.concat([results_loaded, data], axis = 'columns')
        
    data.to_csv('results.csv', index = False)

# Baseline model

В бейзлайн модели будем сравнивать исполнителей на основе количества их треков

In [ ]:
track_data = pd.read_csv('tracks_modified.csv')
event_data = pd.read_csv('events_modified.csv')

In [ ]:
similarities = track_data['artist_id'].value_counts().sort_values(ascending = False).index

In [ ]:
train_events, test_events = traintest_split(event_data)
test_events = test_events.merge(track_data, on = 'track_id')
sampled_users = select_users(event_data)

In [ ]:
all_results = []
for user in sampled_users:
    user_test_data = test_events[test_events['user_id'] == user]
    user_test_artists = user_test_data['artist_id'].unique()
    
    positions = np.isin(similarities, user_test_artists)
    positions = np.where(positions == True)
    result = np.mean(positions[0])/len(similarities)
    all_results.append(result)

In [ ]:
print('Model result:' , np.mean(all_results))

In [ ]:
pd.DataFrame({'baseline_results' : all_results}).to_csv('baseline_results.csv', index = False)

Таким образом, результат baseline модели - 0.179, то есть в среднем, прослушанные во время тестового периода исполнители находятся в топ 17% ранжированного списка

# Advanced models

В advanced моделях будем использовать информацию о прослушиваниях треков артистов из различных датасетов (прослушивания, плейлисты, сессии)

## Events

In [ ]:
event_data = pd.read_csv('events_modified.csv')
track_data = pd.read_csv('tracks_modified.csv')

In [ ]:
train_events, test_events = traintest_split(event_data)
sampled_users = select_users(event_data)

In [ ]:
train_data = count_agg(train_events, track_data, 'user_id', 'playtime', 'sum') 
train_data = add_idx(train_data, 'user_id')

test_data = count_agg(test_events, track_data, 'user_id', 'playtime', 'sum')
test_data = add_idx(test_data, 'user_id')

train_matrix = creating_matrix(train_data, 'user_id_matrix', 'stat')

In [ ]:
matrix_id = train_data[['artist_id', 'artist_id_matrix']]

In [ ]:
del train_events, test_events

In [ ]:
for metric in ['cosine', 'manhattan', 'euclidean']:
    print(metric)
    if metric != 'cosine':
        print('Not cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, metric, inverse = True)
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'events_mod_tracks_mod_total_playtime_{}'.format(metric))
    elif metric == 'cosine':
        print('Actually cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, 'cosine')
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'events_mod_tracks_mod_total_playtime_cosine')

## Sessions

In [ ]:
event_data = pd.read_csv('events_modified.csv')
track_data = pd.read_csv('tracks_modified.csv')
session_data = pd.read_csv('sessions_modified.csv')

In [ ]:
train_events, test_events = traintest_split(event_data)
sampled_users = select_users(event_data)

In [ ]:
train_data = count_agg(train_events, track_data, 'user_id', 'playtime', 'sum')
train_data = add_idx(train_data, 'user_id')

test_data = count_agg(test_events, track_data, 'user_id', 'playtime', 'sum')
test_data = add_idx(test_data, 'user_id')

In [ ]:
train_session, test_session = traintest_split(session_data)

In [ ]:
train_session = count_agg(train_session, track_data, 'session_id', 'playtime', 'count')
train_session = add_idx(train_session, 'session_id')
train_matrix = creating_matrix(train_session, 'session_id_matrix', 'stat')
matrix_id = train_session[['artist_id', 'artist_id_matrix']]

In [ ]:
del train_events, test_events, train_session, test_session

In [ ]:
for metric in ['cosine', 'manhattan', 'euclidean']:
    print(metric)
    if metric != 'cosine':
        print('Not cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, metric, inverse = True)
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'sessions_mod_tracks_mod_total_playtime_{}'.format(metric))
    elif metric == 'cosine':
        print('Actually cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, 'cosine')
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'sessions_mod_tracks_mod_total_playtime_cosine')

## Playlists

In [ ]:
event_data = pd.read_csv('events_modified.csv')
track_data = pd.read_csv('tracks_modified.csv')
playlist_data = pd.read_csv('playlists.csv')

In [ ]:
train_events, test_events = traintest_split(event_data)
sampled_users = select_users(event_data)

In [ ]:
train_data = count_agg(train_events, track_data, 'user_id', 'event_id', 'count')
train_data = add_idx(train_data, 'user_id')

test_data = count_agg(test_events, track_data, 'user_id', 'event_id', 'count')
test_data = add_idx(test_data, 'user_id')

In [ ]:
train_playlist, test_playlist= traintest_split(playlist_data)

In [ ]:
train_playlist = count_agg(train_playlist, track_data, 'playlist_id', 'user_id', 'count')
train_playlist = add_idx(train_playlist, 'playlist_id')
train_matrix = creating_matrix(train_playlist, 'playlist_id_matrix', 'stat')
matrix_id = train_playlist[['artist_id', 'artist_id_matrix']]

In [ ]:
del train_events, test_events, train_playlist, test_playlist

In [ ]:
for metric in ['cosine', 'manhattan', 'euclidean']:
    print(metric)
    if metric != 'cosine':
        print('Not cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, metric, inverse = True)
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'playlists_tracks_mod_count_{}'.format(metric))
    elif metric == 'cosine':
        print('Actually cosine')
        all_metrics, sampled_users, num_train_artists, num_test_artists = predict_similar(sampled_users, train_data, test_data, train_matrix, matrix_id, 'cosine')
        add_results(all_metrics, sampled_users, num_train_artists, num_test_artists, 'playlists_tracks_mod_count_cosine')

# Результаты

## Общие выводы

In [ ]:
results = pd.read_csv('results_full.csv', header = [0, 1])

In [ ]:
results[['events_mod_tracks_mod_total_playtime_cosine']]

In [ ]:
mean_results = results.xs('scores', level=1, drop_level=False, axis = 1).mean().sort_values().droplevel(1)

In [ ]:
_ = plt.figure(figsize = (10, 10))
_ = sns.barplot(y = mean_results.index, x = mean_results.values)
_ = plt.xticks(rotation='45')
_ = plt.title('Среднее значение метрики для моделей (меньше - лучше)')

Как видно, на данной задаче лучше всего себя показала модель, рассчитывающая схожесть с помощью косинусного расстояния на основе информации о прослушиваниях треков различных исполнителей различными пользователями

В целом на большинстве векторных представлений лучший результат показывала метрика расстояния - косинусная близость. Это связано с тем, что в данной задаче скорее важны соотношения между количеством треков в плейлисте/сессии/у пользователя разных исполнителей, а не их абсолютное значение. Косинусное расстояние между, например, точкой (1, 1) и (10, 10) равно нулю, в то время как евклидова метрика и расстояние манхэттена в данном случае отличны от нуля.

Метрики близости, рассчитанные на основе данных о прослушиваниях пользователями треков исполнителей показала лучшие результаты, потому что данное векторное представление наилучшим образом предпочтения пользователей.

Важное дополнение: на одном из этапов работы результаты работы моделей отценивались как среднее место прослушанных треков в ранжированном списке, однако это приводило к неправильной трактовке резултатов: модель основанная на данных о плейлистах давала слишком хорошие результаты, эжто происходило потому, что в данных о плейлистах содержится информация о треках всего 33705 уникальных исполнителей, в то время как в данных о сессиях и прослушиваниях содержатся треки 401415 и 347523 исполнителей соотвественно. Скорее всего исполнители, треки которых есть в плейлистах, в целом являются более актуальными и популярными, что также сказывалось на результатах модели, однако эти результаты не показывали реальной точности модели.

In [ ]:
event_data = pd.read_csv('events_modified.csv')
track_data = pd.read_csv('tracks_modified.csv')
session_data = pd.read_csv('sessions_modified.csv')
playlist_data = pd.read_csv('playlists.csv')

In [ ]:
print('Количество уникальных исполнителей в плейлистах: ', len(pd.merge(playlist_data, track_data, on = 'track_id', how = 'inner')['artist_id'].unique()))
print('Количество уникальных исполнителей в списке прослушиваний: ', len(pd.merge(event_data, track_data, on = 'track_id', how = 'inner')['artist_id'].unique()))
print('Количество уникальных исполнителей в сессиях: ', len(pd.merge(session_data, track_data, on = 'track_id', how = 'inner')['artist_id'].unique()))

## Проверка статистической значимости

Проверим статистическую значимость полученных результатов

In [ ]:
data = results['events_mod_tracks_mod_mean_playtime_cosine']['scores'].values
plt.figure(figsize = (10,10))
_ = sns.distplot(data, bins = 30)
_ = plt.axvline(np.nanmean(data), color = 'red', label = 'mean = {}'.format(round(np.nanmean(data), 4)))
_ = plt.axvline(np.nanmedian(data), color = 'green', label = 'median = {}'.format(round(np.nanmedian(data), 4)))
_ = plt.legend(loc="upper left")
_ = plt.title('Распределение результатов для модели events_mod_tracks_mod_mean_playtime_cosine\n variance = {}'.format(np.nanstd(data)))


data = results['events_mod_tracks_mod_count_cosine']['scores'].values
plt.figure(figsize = (10,10))
_ = sns.distplot(data, bins = 30)
_ = plt.axvline(np.nanmean(data), color = 'red', label = 'mean = {}'.format(round(np.nanmean(data), 4)))
_ = plt.axvline(np.nanmedian(data), color = 'green', label = 'median = {}'.format(round(np.nanmedian(data), 4)))
_ = plt.legend(loc="upper left")
_ = plt.title('Распределение результатов для модели events_mod_tracks_mod_count_cosine\n variance = {}'.format(np.nanstd(data)))


data = results['events_mod_tracks_mod_total_playtime_cosine']['scores'].values
plt.figure(figsize = (10,10))
_ = sns.distplot(data, bins = 30)
_ = plt.axvline(np.nanmean(data), color = 'red', label = 'mean = {}'.format(round(np.nanmean(data), 4)))
_ = plt.axvline(np.nanmedian(data), color = 'green', label = 'median = {}'.format(round(np.nanmedian(data), 4)))
_ = plt.legend(loc="upper left")
_ = plt.title('Распределение результатов для модели events_mod_tracks_mod_total_playtime_cosine\n variance = {}'.format(np.nanstd(data)))

Распределения не являются нормальными, так что при оценке статистчиеской значимости будем использовать непараметррические тесты

Проверим, являются ли различия в результатах моделей статистически значимыми

In [ ]:
results_scores = results[['events_mod_tracks_mod_mean_playtime_cosine', 'events_mod_tracks_mod_count_cosine', 'events_mod_tracks_mod_total_playtime_cosine']].xs('scores', level=1, drop_level=False, axis = 1).droplevel(1, axis = 1)

In [ ]:
p_value_frame = pd.DataFrame(index = results_scores.columns, columns = results_scores.columns, dtype = int)

In [ ]:
for column1 in results_scores:
    for column2 in results_scores:
        p_value = stats.mannwhitneyu(results_scores[column1], 
                                     results_scores[column2], 
                                     alternative = 'two-sided')
        p_value_frame.loc[column1, column2] = p_value[1]
        p_value_frame.loc[column2, column1] = p_value[1]

In [ ]:
_ = plt.figure(figsize = (6, 6))
_ = sns.heatmap(p_value_frame, annot=True, cmap='viridis')
_ = plt.xticks(rotation = 45)
_ = plt.title('P-value от теста манна уитни на разность между средними выборок')

Таким образом, модели на основе данных о прослушиваниях треков с использованием косинусной метрики близости дают статистически незначимо различные результаты

Также проверим статистическую значимость различия результатов baseline модели и лучшей advanced модели

In [ ]:
baseline_results = pd.read_csv('baseline_results.csv')

In [ ]:
_ = plt.title('Распределение среднего результата baseline модели')
_ = sns.distplot(baseline_results['baseline_results'].values)

In [ ]:
stats.mannwhitneyu(baseline_results['baseline_results'].values,
                   results_scores['events_mod_tracks_mod_mean_playtime_cosine'].values,
                   alternative = 'two-sided')

P-value = 0.03, таким образом различие в средних значениях резулльтатов baseline и advanced моделей является статистически значимым, т.к. вероятность получить более крайние значения статистики = 0.03

Итого, лучшая модель - подсчет схожести исполнителей на основе данных о прослушанных пользователями исполнителей с помощью косинусной близости. При этом средний результат составил ~0.16, что на 0.34 лучше, чем у модели, рандомно ранжирующей исполнителей, и на ~0.02 лучше, чем у baseline модели. Также важное отличие от baseline модели заключается в том, что baseline модель хорошо ранжирует исполнителей, но показывает такие хорошие результаты лишь потому, что у самых популярных исполнителей (то есть тех, которых слушает множество пользователей) много треков, соотвественно они оказываются близки в верху ранжируемого списка, одна такое ранжирование имеет мало отношения к схожести между исполнителями.

## Результаты: кратко

Лучшая модель - подсчет схожести исполнителей на основе данных о прослушанных пользователями исполнителей с помощью косинусной близости. При этом средний результат составил ~0.16.
Векторное представление: вектор числа прослушиваний/среднего вемени прослушивания/общего вемени прослушивания (результаты этих моделей рразличаются статистически незначимо) пользователями треков каждого отдельного исполнителя

# Примеры похожих исполнителей

In [ ]:
event_data = pd.read_csv('events_modified.csv')
track_data = pd.read_csv('tracks_modified.csv')
person_data = pd.read_csv('persons.csv')

In [ ]:
all_data = count_agg(event_data, track_data, 'user_id', 'event_id', 'count')
all_data = add_idx(all_data, 'user_id')
all_matrix = creating_matrix(all_data, 'user_id_matrix', 'stat')

Выберем 3 исполнителей с id: 443804, 55302, 356929

In [ ]:
print(person_data[person_data['person_id'] == 443804]['artist_name'].values[0])
print(person_data[person_data['person_id'] == 55302]['artist_name'].values[0])
print(person_data[person_data['person_id'] == 356929]['artist_name'].values[0])

In [ ]:
exmpl_artists = [11893, 15230, 24590]

In [ ]:
artist_similarity1 = pairwise_distances(all_matrix, all_matrix[exmpl_artists[0], :], metric = 'cosine')

In [ ]:
top_20_1 = np.argsort(np.concatenate(artist_similarity1))[:20]

In [ ]:
top_20_1 = all_data[all_data['artist_id_matrix'].isin(top_20_1)]['artist_id'].unique()

Таким образом, самые похожие исполнители на исполнителя Irma and The Fascinations:

In [ ]:
person_data[person_data['person_id'].isin(top_20_1)]['artist_name'].reset_index(drop = True)

In [ ]:
artist_similarity2 = pairwise_distances(all_matrix, all_matrix[exmpl_artists[1], :], metric = 'cosine')

In [ ]:
top_20_2 = np.argsort(np.concatenate(artist_similarity2))[:20]

In [ ]:
top_20_2 = all_data[all_data['artist_id_matrix'].isin(top_20_2)]['artist_id'].unique()

Самые похожие исполнители на исполнителя Bongripper

In [ ]:
person_data[person_data['person_id'].isin(top_20_2)]['artist_name'].reset_index(drop = True)

In [ ]:
artist_similarity3 = pairwise_distances(all_matrix, all_matrix[exmpl_artists[2], :], metric = 'cosine')

In [ ]:
top_20_3 = np.argsort(np.concatenate(artist_similarity3))[:20]

In [ ]:
top_20_3 = all_data[all_data['artist_id_matrix'].isin(top_20_3)]['artist_id'].unique()

Самые похожие исполнители на исполнителя The Funky Lowlives

In [ ]:
person_data[person_data['person_id'].isin(top_20_3)]['artist_name'].reset_index(drop = True)